In [ ]:
dir_ = '/mnt/archive/Antonio/data/manta/smdata/2017-04-30/'
fname = dir_ + '03_10e4x_D700uW_A700uW_inplace.hdf5'

In [ ]:
from pathlib import Path

In [ ]:
fname = Path(fname)
assert fname.is_file()

In [ ]:
mlabel = '{}_{}'.format(fname.parts[-2], fname.stem[:2])
mlabel

# Imports

In [ ]:
import os
from pathlib import Path
import numpy as np
from IPython.display import display
from tqdm import tqdm_notebook, tnrange
import pandas as pd

%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_context('notebook', font_scale=1)

import phconvert as phc
phc.__version__

In [ ]:
from fretbursts import *

In [ ]:
sns = init_notebook(fs=8)

In [ ]:
pd.options.display.max_columns = 48
pd.options.display.max_rows = 48

def make_df_bursts(list_of_columns):
    ncols = 48
    assert len(list_of_columns) == ncols
    nrows = max(len(x) for x in list_of_columns)
    columns = np.arange(ncols)
    df = pd.DataFrame(columns=columns, index=np.arange(nrows), dtype=float)
    df.columns.name = 'spot'
    for col, col_data in zip(columns, list_of_columns):
        df.iloc[:len(col_data), col] = col_data
    return df

def make_df_spots(list_of_tuples=None):
    nrows = 48
    df = pd.DataFrame(index=np.arange(nrows))
    if list_of_tuples is None:
        list_of_tuples = []
    for col, col_data in list_of_tuples:
        df[col] = col_data
    return df

# Load Data

In [ ]:
d = loader.photon_hdf5(str(fname), ondisk=True)  # We have plenty of memory on the server
fname

In [ ]:
print(d.description.decode())

In [ ]:
print('Laser power:  %d mW @ 532    %d mW @ 628' % 
      tuple(d.setup['excitation_input_powers']*1e3))

In [ ]:
if d.alternated and 'A_ON' not in d:
    d.add(A_ON=(100, 1950), D_ON=(2150, 4000), offset=0)

In [ ]:
d.alex_period, d.offset

In [ ]:
if d.alternated:
    bpl.plot_alternation_hist(d)

In [ ]:
if d.alternated:
    loader.alex_apply_period(d)

In [ ]:
d.acquisition_duration

In [ ]:
d.calc_bg_cache(bg.exp_fit, time_s=10, tail_min_us='auto', F_bg=1.7)
d.calc_bg_cache(bg.exp_fit, time_s=5, tail_min_us='auto', F_bg=1.7)

In [ ]:
num_time_points = 5

# Timepoints equally distributed along the measurement
time_points = np.round(np.linspace(d.time_min+1, d.time_max-2, num=num_time_points))

for i in time_points:
    dplot(d, timetrace, tmin=i, tmax=i+1, figsize=(24, 8));
    plt.ylim(-100, 100)
    #savefig("%s_timetrace_t=%d-%d.png" % (mlabel, i, i+1))

In [ ]:
dplot(d, timetrace_bg, show_da=d.alternated);

In [ ]:
bg_AexDem = d.bg_from(Ph_sel(Aex='Dem'))
bg_AexAem = d.bg_from(Ph_sel(Aex='Aem'))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)
fig.suptitle("BG A-excitation period")
ax[0].plot(np.array(bg_AexDem).T, color='g', alpha=0.5);
ax[1].plot(np.array(bg_AexAem).T, color='r', alpha=0.5);

In [ ]:
bg_DexDem = d.bg_from(Ph_sel(Dex='Dem'))
bg_DexAem = d.bg_from(Ph_sel(Dex='Aem'))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)
fig.suptitle("BG D-excitation period")
ax[0].plot(np.array(bg_DexDem).T, color='g', alpha=0.5);
ax[1].plot(np.array(bg_DexAem).T, color='r', alpha=0.5);

In [ ]:
d.burst_search(min_rate_cps=50e3, pax=True)

In [ ]:
d.calc_max_rate(m=10, ph_sel=Ph_sel('all'))

phrate_DAexB = make_df_bursts(d.max_rate)
#display(phrate_DAexB.sample(10))
phrate_DAex = (make_df_spots()
                .assign(**{'num_bursts': d.num_bursts})
                .assign(**{'num_nans': [np.isnan(x).sum() for x in d.max_rate]})
                .assign(**{'num_valid': lambda x: x.num_bursts - x.num_nans})
                .assign(**{'valid_fraction': lambda x: 100 * x.num_valid / x.num_bursts})
              )
phrate_DAex.valid_fraction.mean()

In [ ]:
if d.alternated:
    d.calc_max_rate(m=10, ph_sel=Ph_sel(Dex='DAem'), compact=True)

    phrate_DexB = make_df_bursts(d.max_rate)
    #display(phrate_DexB.sample(10))
    phrate_Dex = (make_df_spots()
                    .assign(**{'num_bursts': d.num_bursts})
                    .assign(**{'num_nans': [np.isnan(x).sum() for x in d.max_rate]})
                    .assign(**{'num_valid': lambda x: x.num_bursts - x.num_nans})
                    .assign(**{'valid_fraction': lambda x: 100 * x.num_valid / x.num_bursts})
                 )
    phrate_Dex.valid_fraction.mean()

In [ ]:
if d.alternated:
    d.calc_max_rate(m=10, ph_sel=Ph_sel(Aex='DAem'), compact=True)

    phrate_AexB = make_df_bursts(d.max_rate)
    #display(phrate_AexB.sample(10))
    phrate_Aex = (make_df_spots()
                    .assign(**{'num_bursts': d.num_bursts})
                    .assign(**{'num_nans': [np.isnan(x).sum() for x in d.max_rate]})
                    .assign(**{'num_valid': lambda x: x.num_bursts - x.num_nans})
                    .assign(**{'valid_fraction': lambda x: 100 * x.num_valid / x.num_bursts})
                 )
    phrate_Aex.valid_fraction.mean()

In [ ]:
if d.alternated:
    d.calc_max_rate(m=10, ph_sel=Ph_sel(Aex='Aem'), compact=True)

    phrate_AexAemB = make_df_bursts(d.max_rate)
    #display(phrate_AexB.sample(10))
    phrate_AexAem = (make_df_spots()
                    .assign(**{'num_bursts': d.num_bursts})
                    .assign(**{'num_nans': [np.isnan(x).sum() for x in d.max_rate]})
                    .assign(**{'num_valid': lambda x: x.num_bursts - x.num_nans})
                    .assign(**{'valid_fraction': lambda x: 100 * x.num_valid / x.num_bursts})
                 )
    phrate_AexAem.valid_fraction.mean()

In [ ]:
if d.alternated:
    d.calc_max_rate(m=10, ph_sel=Ph_sel(Aex='Dem'), compact=True)

    phrate_AexDemB = make_df_bursts(d.max_rate)
    #display(phrate_AexB.sample(10))
    phrate_AexDem = (make_df_spots()
                    .assign(**{'num_bursts': d.num_bursts})
                    .assign(**{'num_nans': [np.isnan(x).sum() for x in d.max_rate]})
                    .assign(**{'num_valid': lambda x: x.num_bursts - x.num_nans})
                    .assign(**{'valid_fraction': lambda x: 100 * x.num_valid / x.num_bursts})
                 )
    phrate_AexDem.valid_fraction.mean()

In [ ]:
dplot_48ch(phrate_DexB, hist_burst_phrate);

In [ ]:
if d.alternated:
    dplot_48ch(phrate_AexB, hist_burst_phrate);

In [ ]:
if d.alternated:
    dplot_48ch(phrate_AexDemB, hist_burst_phrate);

In [ ]:
if d.alternated:
    dplot_48ch(phrate_AexAemB, hist_burst_phrate);

In [ ]:
dplot(d, hist_size);

In [ ]:
th1 = 40
ds = d.select_bursts(select_bursts.size, th1=th1, gamma=0.5, add_aex=True)

Su = [(nd + na)/(nd + na + naa) for nd, na, naa in zip(ds.nd, ds.na, ds.naa)]
ds.add(Su=Su)

In [ ]:
dplot(ds, hist_fret, pdf=False, figsize=(18, 6));

In [ ]:
if not d.alternated:
    stop_here

In [ ]:
dplot(ds, hist_S, pdf=False, figsize=(18, 6));

In [ ]:
dplot(ds, hist_burst_data, data_name='Su', pdf=False, figsize=(18, 6));

In [ ]:
kws = dict(bins=np.arange(-0.1, 2.1, 0.025), histtype='step', lw=1.5)
for i in (17, 18, 29, 30):
    plt.hist(ds.S[i], label='CH%d' % i, **kws);
plt.legend()

In [ ]:
kws = dict(bins=np.arange(-0.1, 2.1, 0.025), histtype='step', lw=1.5)
for i in (17, 18, 29, 30):
    plt.hist(ds.Su[i], label='CH%d' % i, **kws);
plt.legend()

In [ ]:
Smin, Smax = 0, 1
jkws = dict(extent=(-0.2 + Smin, Smax + 0.2, -0.2 + Smin, Smax + 0.2), gridsize=80*(Smax-Smin))
mkws = dict(bins=np.arange(-0.2 + Smin, Smax + 0.2, 0.03), show_kde=False)
for i in (17, 18, 29, 30):
    plt.figure()
    g = alex_jointplot(ds, i=i, cmap='viridis', vmax_fret=False, 
                       joint_kws=jkws, marginal_kws=mkws)
    g.ax_joint.set_ylim(-0.2, 1.2)
    g.ax_joint.set_xlim(-0.2, 1.2)

In [ ]:
Smin, Smax = 0, 1
jkws = dict(extent=(-0.2 + Smin, Smax + 0.2, -0.2 + Smin, Smax + 0.2), gridsize=80*(Smax-Smin))
mkws = dict(bins=np.arange(-0.2 + Smin, Smax + 0.2, 0.03), show_kde=False)
for i in (17, 18, 29, 30):
    plt.figure()
    g = alex_jointplot(ds, i=i, cmap='viridis', vmax_fret=False, 
                       joint_kws=jkws, marginal_kws=mkws, S_name='Su')
    g.ax_joint.set_ylim(-0.2, 1.2)
    g.ax_joint.set_xlim(-0.2, 1.2)

In [ ]:
d2 = bext.burst_search_and_gate(d)

In [ ]:
th1 = 40
ds2 = d2.select_bursts(select_bursts.size, th1=th1, gamma=0.5, add_aex=True)

In [ ]:
Su = [(nd + na)/(nd + na + naa) for nd, na, naa in zip(ds2.nd, ds2.na, ds2.naa)]
ds2.add(Su=Su)

In [ ]:
Smin, Smax = 0, 1
jkws = dict(extent=(-0.2 + Smin, Smax + 0.2, -0.2 + Smin, Smax + 0.2), gridsize=80*(Smax-Smin))
mkws = dict(bins=np.arange(-0.2 + Smin, Smax + 0.2, 0.03), show_kde=False)
for i in (17, 18, 29, 30):
    plt.figure()
    g = alex_jointplot(ds2, i=i, cmap='viridis', vmax_fret=False, 
                       joint_kws=jkws, marginal_kws=mkws)
    g.ax_joint.set_ylim(-0.2, 1.2)
    g.ax_joint.set_xlim(-0.2, 1.2)

In [ ]:
Smin, Smax = 0, 1
jkws = dict(extent=(-0.2 + Smin, Smax + 0.2, -0.2 + Smin, Smax + 0.2), gridsize=80*(Smax-Smin))
mkws = dict(bins=np.arange(-0.2 + Smin, Smax + 0.2, 0.03), show_kde=False)
for i in (17, 18, 29, 30):
    plt.figure()
    g = alex_jointplot(ds2, i=i, cmap='viridis', vmax_fret=False, 
                       joint_kws=jkws, marginal_kws=mkws, S_name='Su')
    g.ax_joint.set_ylim(-0.2, 1.2)
    g.ax_joint.set_xlim(-0.2, 1.2)

In [ ]:
dsaa = d.select_bursts(select_bursts.naa, th1=15)
Su = [(nd + na)/(nd + na + naa) for nd, na, naa in zip(dsaa.nd, dsaa.na, dsaa.naa)]
dsaa.add(Su=Su)

In [ ]:
Smin, Smax = 0, 1
jkws = dict(extent=(-0.2 + Smin, Smax + 0.2, -0.2 + Smin, Smax + 0.2), gridsize=80*(Smax-Smin))
mkws = dict(bins=np.arange(-0.2 + Smin, Smax + 0.2, 0.03), show_kde=False)
for i in (17, 18, 29, 30):
    plt.figure()
    g = alex_jointplot(dsaa, i=i, cmap='viridis', vmax_fret=False, 
                       joint_kws=jkws, marginal_kws=mkws, S_name='Su')
    g.ax_joint.set_ylim(-0.2, 1.2)
    g.ax_joint.set_xlim(-0.2, 1.2)